In [4]:
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm


import funcs.ArticlePage as ArticlePage
import funcs.AuthorPage as AuthorPage
import funcs.Common as Common
import networkx as nx
import importlib
import time


In [3]:
firefox_service = Service()
web_driver = webdriver.Firefox(service = firefox_service)
url = "https://www.scopus.com/"
web_driver.get(url)

url_prefix = "https://www.scopus.com/authid/detail.uri?authorId="

In [ ]:
failed_ids=[7005844510,55362161700,9734543400,35315426400,55358691500,10241618100,54976304900,36145857300,6602492239,16070578000,7102536021,57719874400,57024220000,56102265700,57194135400,57217896809,57205443945,6602268525,57219586927,36634617200,57205532529,26531952500,7005034358,36763188600,30567911800,56497195900,7102949760,57214724995,36666074500,57338880900,8857319300,56279109000,57201673097,57194453898,56400610700,57204639118,55752154000,55865706900,7006787992,57212242329,57111671200,8308522400,23396313000,23388121000,7801658282,57204296619,57221141935,36080404400,55557178800,7102731698,36145850800,58117413300,57117489600,6603156928,57203025864,55540823500,26532123600,7003967440,57754032600,57207692762,7102149594,57209344990,23969068000,6508172257,56389488100,24823924200,15032551400,7102447605,55787127800,7003815417,57161275900]
print(len(failed_ids))

In [6]:
importlib.reload(ArticlePage)
_ = importlib.reload(AuthorPage)


In [ ]:
current_layer = set(failed_ids)
author_graph = nx.Graph()
explored_ids = set()
next_layer = set()

MAX_TRIES = 5

for author_id in current_layer:
	explored_ids.add(author_id) #Adiciona autor aos visitados
	
	attempts = 0
	success = False
	while not success and attempts < MAX_TRIES: #Loop de tentativas para lidar com erros de carregamento (404, 500, etc.)
		try:
			web_driver.get(url_prefix + str(author_id)) #Acessa página do autor
			time.sleep(4)
			AuthorPage.add_author(web_driver, author_id, author_graph) #Adiciona autor e dados ao grafo
			success = True
		except KeyboardInterrupt:
			raise
		except:
			attempts+=1

	if(not success): #Lida com falha de obter autor
		print(f"Failed Author for {author_id}")
		try:
			soup=BeautifulSoup(web_driver.page_source,'lxml')
			file=open(f"Fails\\Author_{author_id}.html", "w", encoding="utf-8")
			file.write(f"Author: {author_id}\n\n")
			file.write(str(soup))
			file.close()
		except KeyboardInterrupt:
			raise
		except:
			pass
		continue #Pula autor

	articles = AuthorPage.get_author_articles(web_driver, sleep_time=4) #Lista de artigos do pesquisador

	for article_url in articles:
		attempts = 0
		success = False
		while not success and attempts < MAX_TRIES: #Loop de tentativas para lidar com erros de carregamento (404, 500, etc.)
			try:
				web_driver.get(article_url) #Acessa página do Artigo
				time.sleep(4)
				ArticlePage.add_article(web_driver, explored_ids, next_layer, author_graph, author_id) #Adiciona conexões formadas pelo artigo
				success = True
			except KeyboardInterrupt:
				raise
			except:
				attempts+=1
			
			if(not success): #Lida com falha de obter artigo
				print(f"Failed Article for {author_id}")
				try:
					soup=BeautifulSoup(web_driver.page_source,'lxml')
					file=open(f"Fails\\Article_{author_id}.html", "w", encoding="utf-8")
					file.write(f"Author: {author_id}\n\n")
					file.write(str(soup))
					file.close()
				except KeyboardInterrupt:
					raise
				except:
					pass


Common.play_notification()

In [ ]:
name = f"../gmls/gml_fails.gml"
nx.write_gml(author_graph, str(name))

In [29]:
web_driver.quit()